# 17. Convert Detection Log to GeoJSON

Given a detection log from applying the model to dashcam images in a survey area (filtered or unfiltered) align the detections to the OpenStreetMap data and produce geojson files to map and compare the output

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Name of the subdirectory containing dashcam footage for an area, split into frame images in a
# "split" subdirectory, with an associated "metadata.csv"
# This subdirectory is assumed to be in the 'data_sources' directory
import_directory = 'dashcam_tour_mount_eliza'

# Which "locality" do we wish to process?
locality = 'Mount Eliza'

# We will sample the middle of each intersection, but we can also sample a
# "margin" around the intersection, at 10m intervals.
# E.g. if we set this to "20" then we will sample points at:
#    -20m, -10m, 0m, 10m, and 20m
# from the centre of the intersection, along the assumed bearing of the road
# Used here just to get the right filename
margin = 20

# Name of the detection log filename
filename_in  = 'detection_log_filtered.csv'

# Name of the output detection log that has been aligned to the OpenStreetMap data
filename_out = 'detection_log_filtered_snap.csv'

## Code

In [2]:
# General imports
import os
import sys

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Import local modules
import osm_gsv_utils.osm_walker as osm_walker
import osm_gsv_utils.detection_log_filter as detection_log_filter

In [3]:
# Derive paths for configuration

# A version of the locality name with spaces replaced by underscores
locality_clean = locality.replace(' ', '_')

# Work out paths to OSM data
filename_main        = os.path.join(module_path_root, 'data_sources', 'Locality_' + locality_clean + '.osm')
filename_margin      = os.path.join(module_path_root, 'data_sources', 'Locality_' + locality_clean + '_margin.osm')
locality_margin      = '{0:s}_{1:d}m'.format(locality_clean, margin)

# Work out path to the detection log input and the place we will write the filtered version
detections_dir       = os.path.join(module_path_root, 'data_sources', import_directory, 'detections')
detections_path_in   = os.path.join(detections_dir, filename_in)
detections_path_out  = os.path.join(detections_dir, filename_out)

# Metadata CSV containing ALL points that were sampled in the dashcam footage
metadata_path        = os.path.join(module_path_root, 'data_sources', import_directory, 'split', 'metadata.csv')

# Metadata CSV converted to detection_log format
route_path_converted = os.path.join(detections_dir, 'route_log.csv')

# Route data aligned to OSM data
route_path_snapped   = os.path.join(detections_dir, 'route_log_snapped.csv')

# Filtered OSM datafile containing just the route
filename_route_osm   = os.path.join(detections_dir, 'route.osm')

## Processing the Route data

If we want to compare the detected routes to OpenStreetMap, we want to limit the comparison to only the routes that were surveyed in the footage.  Otherwise, we might expect OpenStreetMap to return a lot more routes, and the comparison would be meaningless.

So, first we want to take the metadata about the footage collected, and convert it into the "detection log" format, so we can use one function in the "osm_walker" class to align either the route metadata points or the detection points to the OpenStreetMap route data.

In [4]:
# Load OSM data for the broad area around the survey area
walker1 = osm_walker(filename_margin, None, verbose=False)

# Make sure that any ways that were split into multiple segments due to a changed
# characteristic (e.g. speed limit) are linked back up again
walker1.link_way_sections(verbose=False)

  0%|          | 0/4274 [00:00<?, ?it/s]

In [5]:
# Convert route metadata for collected footage to detection_log format
detection_log_filter.convert_metadata_to_route_log(metadata_path, route_path_converted)

  0%|          | 0/13545 [00:00<?, ?it/s]

In [6]:
# Use the OSM data for the general area around the survey area to align the
# points in the route metadata to OSM routes
walker1.snap_detection_log(route_path_converted, route_path_snapped)

  0%|          | 0/13545 [00:00<?, ?it/s]

## Align detections to OSM data for route

Align the detections to the OSM data that includes the route only, compare to cycleway routes, output comparison statistics and geojson files that can be drawn on a map.

In [7]:
# Reload OSM data, this time applying a filter so that only nodes that were passed by the survey route are included
walker2 = osm_walker(filename_margin, None, filter_log=route_path_snapped, verbose=False)

# Make sure that any ways that were split into multiple segments due to a changed
# characteristic (e.g. speed limit) are linked back up again
walker2.link_way_sections(verbose=False)

  0%|          | 0/20467 [00:00<?, ?it/s]

  0%|          | 0/4274 [00:00<?, ?it/s]

Nodes Loaded: [602] Filtered = [30155]


In [8]:
# Snap route of collected footage to nearest intersections
walker2.snap_detection_log(detections_path_in, detections_path_out)

  0%|          | 0/753 [00:00<?, ?it/s]

In [9]:
# Load the snapped detection log
walker2.load_detection_log(detections_path_out)

In [10]:
# Write GeoJSON files, including comparison files
walker2.write_geojsons(locality_margin, detections_dir, intersection_skip_limit=2, infer_ends=False, verbose=False)

Writing hit, feature count: 20
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\hit.geojson
Writing tag, feature count: 10
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\tag.geojson
Writing both, feature count: 8
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\both.geojson
Writing either, feature count: 22
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\either.geojson
Writing hit_only, feature count: 14
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\hit_only.geojson
Writing tag_only, feature count: 4
Writing to: E:\Release\minor_thesis\data_sources\dashcam_tour_mount_eliza\detections\tag_only.geojson
hit     : Total distance   13413.24m
tag     : Total distance    8902.82m
both    : Total distance    7483.63m
either  : Total distance   14832.43m
hit_only: Total distance    5795.54m
tag_only: Total dista